# 7. Stitch together the results
At this point in the tutorial we should expect to see the output images of the style transfer inside your specified blob container. Inside the same container, we would also be able to inspect the logs for each job. 

In this notebook, we'll wrap the tutorial by downloading the output images of the video style transfer job and stitching the newly generated style transfered images back into a video.

```md
pytorch
├── images/
│   ├── orangutan/
│   ├── output_orangutan_MM_DD_YYYY_HHMMSS/ [<-- this folder of newly generated images will be downloaded from storage]
│   ├── sample_content_images/
│   ├── sample_output_images/
│   └── style_images/
├── video/
│   ├── orangutan_processed_without_audio.mp4 [<-- this video file will be generated from the individual frames]
│   ├── orangutan_processed_with_audio.mp4 [<-- this video file is created by adding audio back]
│   ├── orangutan.mp4
│   └── orangutan.mp3
├── style_transfer_script.py
└── style_transfer_script.log
```

After running the tutorial, your working directory under `/pytorch` should look as shown above with the newly generated files.

---

In [ ]:
import os
from IPython.display import HTML
%load_ext dotenv
%dotenv

First we need to get the name of the output folder where all out style transfered frames are stored. We do this by using the Batch AI CLI. We store the name of the output folder as `output_folder_name` to be used in later cells.

In [ ]:
# We use `--query [-1].name` to mean: 'get the name of the last experiment'
exp_names = !az batchai experiment list -g ${RESOURCE_GROUP} --workspace ${WORKSPACE} --query [-1].name

# strip quotation marks
exp_name = str(exp_names[0])[1:-1]

# replace experiment prefix with output folder prefix
exp_name_array = str(exp_name).split('_')
exp_name_array[0] = "{0}_{1}".format(os.getenv('FS_OUTPUT_DIR_PREFIX'), os.getenv('FS_CONTENT_DIR'))
output_folder_name = '_'.join(exp_name_array)

Using the `output_folder_name` variable, lets download the frames:

In [ ]:
%%script env output_folder_name="$output_folder_name" bash
cd pytorch/images &&
    azcopy \
        --source https://${STORAGE_ACCOUNT_NAME}.blob.core.windows.net/${AZURE_CONTAINER_NAME} \
        --destination . \
        --source-key $STORAGE_ACCOUNT_KEY \
        --include $output_folder_name \
        --recursive

Then lets use __ffmpeg__ to stitch independent frames back into a video.

In [ ]:
%%script env output_folder_name="$output_folder_name" bash
cd pytorch &&
    ffmpeg \
        -framerate 30 \
        -i images/$output_folder_name/%05d_${VIDEO_NAME}.jpg \
        -c:v libx264 \
        -profile:v high -crf 20 -pix_fmt yuv420p \
        -y \
        video/${VIDEO_NAME}_processed_without_audio.mp4 

In the beginning, we also used __ffmpeg__ to extract out the original audio file. Now that we have our newly generated video, attach the audio file to the new video.

In [ ]:
%%bash
cd pytorch/video &&
    ffmpeg -i ${VIDEO_NAME}_processed_without_audio.mp4 -i ${VIDEO_NAME}.mp3 \
        -map 0:0 -map 1:0 \
        -vcodec copy -acodec copy \
        -y \
        ${VIDEO_NAME}_processed_with_audio.mp4

Lets take a look at the final result

In [ ]:
HTML('\
    <video width="360" height="360" controls> \
         <source src="pytorch/video/{0}_processed_with_audio.mp4" type="video/mp4"> \
    </video>'\
    .format(os.getenv('VIDEO_NAME'))
)

---

## Conclusion
In this notebook, we downloaded the individually generated frames from the style transfer job back to our local machine and stitched it back together using __ffmpeg__. Now we have our neural style transferred video fully generated!

Congratulations, you've finished the tutorial and now have an awesome video to share with friends. 

To wrap up, lets clean up our resources. Because we created everything in a single resource group, we can just delete that resource group to clean up all resources in Azure. 

Throughout this tutorial, we've also generated a lot of files in our working directory. To restore this directory to its original state, you can use the file `/cleanup_files.sh`.

Delete your resource group:

In [4]:
%load_ext dotenv
%dotenv
! az group delete --name $RESOURCE_GROUP -y

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
Resource group 'js_batch_scoring_dl' could not be found.


Run the `.cleanup_files` script

In [ ]:
./cleanup_files.sh